In [1]:
%load_ext autoreload
%autoreload 2
import importlib
from sklearn.ensemble import RandomForestClassifier
import minirocket_multivariate_variable as mmv
importlib.reload(mmv)
from sklearn.linear_model import LogisticRegression
from utils import get_cognitive_circles_data_for_classification
from classifier import MinirocketClassifier

/home/lgalarra/Documents/git/xai-for-minirocket/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Fetching data

In [2]:
(X_train, y_train), (X_test, y_test) = get_cognitive_circles_data_for_classification('../data/cognitive-circles', target_col='RealDifficulty', as_numpy=True)

## Training MiniROCKET

In [4]:
%load_ext autoreload
%autoreload 2
#mr_classifier = LogisticRegression(max_iter=2000, solver="lbfgs", multi_class="auto")
mr_classifier = RandomForestClassifier(n_estimators=100)
classifier = MinirocketClassifier(minirocket_features_classifier=mr_classifier)
classifier.fit(X_train, y_train)
from sklearn.metrics import accuracy_score
print('Accuracy score:', accuracy_score(y_test, classifier.predict(X_test)))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Accuracy score: 0.75


## Explaining two instances

In [5]:
explainer =  classifier.get_explainer()
explanation = explainer.explain_instances(X_test[0:2], classifier_explainer='shap', reference_policy='opposite_class_centroid')
print(explanation.explanation[0]['coefficients'])

ValueError: operands could not be broadcast together with shapes (7874,7) (7,1,7874) (7874,7) 